In [ ]:
import json
import jellyfish
import nltk
import statistics
from collections import Counter
from tqdm import tqdm

# Load KWX

In [ ]:
kwx = json.load(open('./data.json','r'))
train_size = 16000

# Metrics

In [ ]:
def compare_two_words(w1,w2):
    return jellyfish.jaro_distance(w1, w2)

def is_related_in_list(word,ys):
    for y in ys:
        if compare_two_words(word.lower(),y.lower()) >= 0.9:
            return True
    return False 

def kws_precision(ys,ys_true):
    if len(ys) >= 1:
        count = 0 
        for y in ys:
            if is_related_in_list(y,ys_true):
                count += 1
        return count/len(ys)
    else:
        return 0

def kws_recall(ys,ys_true):
    if not ys_true:
        return 0
    if len(ys) >= 1:
        count = 0 
        for y in ys:
            if is_related_in_list(y,ys_true):
                count += 1
        if count > len(ys_true):
            return 1
        return count/len(ys_true)
    else:
        return 0 
    
def evaluate_kws(ys,ys_true):
    res = {}
    res['precision'] = kws_precision(ys,ys_true)
    res['recall'] = kws_recall(ys,ys_true)
    if res['precision'] or res['recall']:
        res['f1'] = (2*res['precision']*res['recall'])/(res['precision']+res['recall'])
    else:
        res['f1'] = 0
    return res 


def macro_pr(yt_pairs):
    f1 = []
    p = []
    r = []
    for pair in yt_pairs:
        evaluate = evaluate_kws(pair[0],pair[1])
        f1.append(evaluate['f1'])
        p.append(evaluate['precision'])
        r.append(evaluate['recall'])
    return {'f1':statistics.mean(f1),
           'precision':statistics.mean(p),
           'recall':statistics.mean(r)}

def evaluate_extractor(docs_with_kws, extractor, top=None):
    yt_pairs = []
    for i in tqdm(list(docs_with_kws.keys())[train_size:]):
        pair = []
        pair.append(extractor(docs_with_kws[i]['abstract'],top))
        ys_true = docs_with_kws[i]['keywords']
        if 'keywords_extra' in docs_with_kws[i]:
            ys_true += docs_with_kws[i]['keywords_extra']
        pair.append(ys_true)
        yt_pairs.append(pair)
    print(len(yt_pairs))
    print(macro_pr(yt_pairs))

# Keyword Extraction

In [ ]:
kw_start = [
    'NN',
    'NNP',
    'JJ',
    'NNS',
    'VBG',
    'RB',
    'VBN'
]

kw_end = [
    'NN',
    'NNP',
    'NNS',
    'VBG',
    'JJ'
]

kw_split = [
    '.',
    ','
]
def keywords_selector(text):
    tokens = nltk.word_tokenize(text)
    tags = nltk.pos_tag(tokens)
    res = set()
    for i in range(len(tags)):
        if tags[i][1] in kw_start:
            end = i+4 if i+4 <= len(tags) - 1 else len(tags) - 1
            cut = tags[i:end]
            for j in range(len(cut)):
                if cut[j][0] in kw_split:
                    cut = cut[:j]
                    break 
            for k in range(len(cut)):
                if cut[k][1] in kw_end:
                    res.add(' '.join([m[0] for m in cut][:k+1]))
    return res 
    

In [ ]:
# scoreboard to record all combinations.
history_score = {}
# global dictionary to record all links between present and absent keywords
global_kwx_index = {}
for i in tqdm(list(kwx.keys())[:train_size]):
    ys = kwx[i]['keywords']
    candidates = keywords_selector(kwx[i]['abstract'])
    for c in candidates:
        if not c in history_score:
            history_score[c] = 0
        if is_related_in_list(c,ys):
            history_score[c] += 100
        else:
            history_score[c] -= 1
    for y in ys:
        if y in global_kwx_index:
            global_kwx_index[y].update(ys)
        else:
            global_kwx_index[y] = Counter(ys)
print(len(history_score))

In [ ]:
def kwx_kws(text,top=None):
    res = []
    score = []
    candidates = keywords_selector(text)
    for c in candidates:
        if c not in history_score:
            res.append(c)
            score.append(1)
        elif history_score[c]>0:
            res.append(c)
            score.append(history_score[c])
        if c in global_kwx_index:
            for r in global_kwx_index[c]:
                if r not in candidates:
                    res.append(r)
                    score.append(global_kwx_index[c][r])
    if not res:
        res = ['None']
    sorted_res = [x for _, x in sorted(zip(score, res))][::-1]
    if top and len(sorted_res)>top:
        return sorted_res[:top]
    else:
        return sorted_res

In [ ]:
evaluate_extractor(kwx, kwx_kws)

In [ ]:
evaluate_extractor(kwx, kwx_kws, 10)